## Dependencies

#download the data
http://xkcd.com/color/colorsurvey.tar.gz

In [0]:
import random
import argparse

In [0]:
%%bash
tar -xvzf data/colorsurvey.tar.gz -C data

mainsurvey_sqldump.txt
satfaces_sqldump.txt


In [0]:
ls data

colorsurvey.tar.gz  mainsurvey_sqldump.txt  satfaces_sqldump.txt  train.csv
_DS_Store           mytest.csv              test.csv


In [0]:
all_fields = []
with open('data/mainsurvey_sqldump.txt') as f:
    for line in f:
        if '"answers"' not in line:
            continue
        
        fields = line[:-3].split(',')[3:7]
        fields[-1] = fields[-1].strip("'")
        all_fields.append(fields)
        
random.shuffle(all_fields)

In [0]:
split = int(0.9*len(all_fields))
with open('data/train.csv','w') as out_file:
    out_file.write("red,green,blue,name\n")
    for fields in all_fields[:split]:
        out_file.write(','.join(fields)+"\n")


In [0]:
with open('data/test.csv','w') as out_file:
    out_file.write("red,green,blue,name\n")
    for fields in all_fields[split:]:
        out_file.write(','.join(fields)+"\n")

In [0]:
%%bash
head data/test.csv

red,green,blue,name
121,107,126,grey
183,152,160,rosa opaco
25,61,200,blue
2,194,213,light blue
75,127,37,moss
18,73,38,hunter green
63,181,161,green
82,241,222,droopy
59,43,186,vibrant blue


In [0]:
%%bash
head data/test.csv > data/mytest.csv

In [0]:
del all_fields

In [0]:

# Tensorflow
import tensorflow as tf

# Feeding function for enqueue data
# 
from tensorflow.python.estimator.inputs.queues import feeding_functions as ff

# Rnn common functions
from tensorflow.contrib.learn.python.learn.estimators import rnn_common

# Run an experiment
from tensorflow.contrib.learn.python.learn import learn_runner

# Input function
from tensorflow.python.estimator.inputs import numpy_io

# Helpers for data processing
import pandas as pd
import numpy as np

# Plot images with pyplot
%matplotlib inline
from matplotlib import pyplot as plt

## Parameters

In [0]:
# Data files
TRAIN_INPUT = 'data/train.csv'
TEST_INPUT = 'data/test.csv'
MY_TEST_INPUT = 'data/mytest.csv'

# Parameters for training
STEPS = 10000
BATCH_SIZE = 64

# Parameters for data processing
CHARACTERS = [chr(i) for i in range(256)]

SEQUENCE_LENGTH_KEY = 'sequence_length'
COLOR_NAME_KEY = 'color_name'
RGB_KEY = 'rgb'

## Helper functions

In [0]:
# This function creates a sparse tensor in the following way, given:
# indices = [[0, 0], [1, 1], [2, 2]]
# values = [1, 2, 3]
# dense_shape = [3, 4]
#
# The output will be a sparse tensor that represents this dense tensor:
# [ 
#   [1, 0, 0, 0]
#   [0, 2, 0, 0]
#   [0, 0, 3, 0]
# ]
#
# We're using this to generate a Sparse tensor that can be easily
# formated in a one hot representation.
# More at: https://www.tensorflow.org/api_docs/python/tf/SparseTensor
def _sparse_string_to_index(sp, mapping):
    table = tf.contrib.lookup.index_table_from_tensor(mapping, dtype=tf.string)
    
    return tf.SparseTensor(indices=sp.indices,
                           values=table.lookup(sp.values),
                           dense_shape=sp.dense_shape)

# Returns the column values from a CSV file as a list
def _get_csv_column(csv_file, column_name):
    with open(csv_file, 'r') as f:
        df = pd.read_csv(f)
        return df[column_name].tolist()

# Plot a color image
def _plot_rgb(rgb):
    data = [[rgb]]
    plt.figure(figsize=(2,2))
    plt.imshow(data, interpolation='nearest')
    plt.show()

## Input functions

In [0]:
df = pd.read_csv('data/train.csv')

In [0]:
len(df)

2982631

In [0]:
length = df.name.str.len()
length.quantile(0.9999)

99.0

In [0]:
class ColorsDataset(object):
    def __init__(self, csv_file, batch_size, epochs=1):
        self.csv_file = csv_file
        self.batch_size = batch_size
        self.epochs = epochs

        df = pd.read_csv(csv_file)
        df.red = np.int32(df.red)
        df.green = np.int32(df.green)
        df.blue = np.int32(df.blue)
        df = df.dropna()
        
        # Sequence length is used by the Dynamic RNN
        # to dynamically unroll the graph :D!
        df['sequence_length'] = df.name.str.len().astype(np.int32)
        df = df[df.sequence_length < df.sequence_length.quantile(0.9999)]

        self.df = df        
        self.pd_fn = tf.estimator.inputs.pandas_input_fn(
            self.df, shuffle=True, 
            batch_size=self.batch_size, 
            num_epochs=self.epochs,
            queue_capacity=100000)

    def __call__(self):
        fields = argparse.Namespace(**self.pd_fn())

        # Split strings into chars
        split_color_name = tf.string_split(fields.name, delimiter='')
        # Creating a tf constant to hold the map char -> index
        # this is need to create the sparse tensor and after the one hot encode
        table = tf.contrib.lookup.index_table_from_tensor(CHARACTERS, dtype=tf.string)

        # Names represented in a sparse tensor
        integerized_color_name = table.lookup(split_color_name)
        
        print(integerized_color_name)
        
        # Tensor of normalized RGB values
        rgb = tf.to_float(tf.stack([fields.red, fields.green, fields.blue], axis=1)) / 255.0

        features = {COLOR_NAME_KEY: integerized_color_name, SEQUENCE_LENGTH_KEY: fields.sequence_length}
        return features, rgb


In [0]:
# Creating my own input function for a custom CSV file
# it's simpler than the input_fn above but just used for small tests

def my_test_input_fn():
    df = pd.read_csv(MY_TEST_INPUT)
    df = df.dropna()
    df['sequence_length'] = df.name.str.len().astype(np.int32)

    color_name = df.name.tolist()

    split_color_name = tf.string_split(color_name, delimiter='')
    mapping = tf.constant(CHARACTERS, name="mapping")
    integerized_color_name = _sparse_string_to_index(split_color_name, mapping)

    x = {COLOR_NAME_KEY: integerized_color_name, SEQUENCE_LENGTH_KEY: df.sequence_length.tolist()}

    y = np.asarray([[0, 0, 0]], dtype=np.float32)

    return x, y
    

In [0]:
train_input_fn = ColorsDataset(TRAIN_INPUT, BATCH_SIZE)
test_input_fn = ColorsDataset(TEST_INPUT, BATCH_SIZE)

In [0]:
'''
# Testing the input function
with tf.Graph().as_default():
    train_input = train_input_fn()
    with tf.train.MonitoredSession() as sess:
        print (train_input)
        print (sess.run(train_input))
'''

'\n# Testing the input function\nwith tf.Graph().as_default():\n    train_input = train_input_fn()\n    with tf.train.MonitoredSession() as sess:\n        print (train_input)\n        print (sess.run(train_input))\n'

## Creating the Estimator model

In [0]:
def get_model_fn(rnn_cell_sizes,
                 label_dimension,
                 dnn_layer_sizes=[],
                 optimizer='SGD',
                 learning_rate=0.01):
    
    def model_fn(features, labels, mode):
        color_name = features[COLOR_NAME_KEY]
        sequence_length = features[SEQUENCE_LENGTH_KEY]

        # Creating dense representation for the names
        # and then converting it to one hot representation
        dense_color_name = tf.sparse_tensor_to_dense(color_name)
        e_layer = tf.contrib.keras.layers.Embedding(len(CHARACTERS), 30)
        embedded = e_layer(dense_color_name)
        color_name_onehot = embedded
        #color_name_onehot = tf.one_hot(dense_color_name, depth=len(CHARACTERS) + 1)
        
        
        # Each RNN layer will consist of a LSTM cell
        rnn_layers = [tf.nn.rnn_cell.BasicLSTMCell(size) for size in rnn_cell_sizes]
        
        # Construct the layers
        multi_rnn_cell = tf.contrib.rnn.MultiRNNCell(rnn_layers)
        
        # Runs the RNN model dynamically
        # more about it at: 
        # https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn
        outputs, final_state = tf.nn.dynamic_rnn(cell=multi_rnn_cell,
                                                 inputs=color_name_onehot,
                                                 sequence_length=sequence_length,
                                                 dtype=tf.float32)

        # Slice to keep only the last cell of the RNN
        last_activations = rnn_common.select_last_activations(outputs,
                                                              sequence_length)

        # Construct dense layers on top of the last cell of the RNN
        for units in dnn_layer_sizes:
            last_activations = tf.layers.dense(
              last_activations, units, activation=tf.nn.relu)
        
        # Final dense layer for prediction
        predictions = tf.layers.dense(last_activations, label_dimension, activation=tf.nn.sigmoid)

        loss = None
        train_op = None

        if mode != tf.contrib.learn.ModeKeys.INFER:    
            loss = tf.losses.mean_squared_error(labels, predictions)
    
        if mode == tf.contrib.learn.ModeKeys.TRAIN:  
            global_step = tf.contrib.framework.get_global_step()
        
            train_op = tf.contrib.layers.optimize_loss(
              loss,
              global_step,
              optimizer=optimizer,
              learning_rate=learning_rate)
                
            
        
        return tf.estimator.EstimatorSpec(mode,
                                          predictions=predictions,
                                          loss=loss,
                                          train_op=train_op)
    return model_fn

In [0]:
model_fn = get_model_fn(rnn_cell_sizes=[256, 128], # size of the hidden layers
                        label_dimension=3, # since is RGB
                        dnn_layer_sizes=[64], # size of units in the dense layers on top of the RNN
                        optimizer='Adam',
                        learning_rate=0.01)
estimator = tf.estimator.Estimator(model_fn=model_fn, model_dir='./checkpoints/c/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_model_dir': './checkpoints/c/', '_save_summary_steps': 100}


In [0]:
for n in range(10):
    estimator.train(input_fn=train_input_fn)
    estimator.evaluate(input_fn = test_input_fn)

SparseTensor(indices=Tensor("StringSplit:0", shape=(?, 2), dtype=int64, device=/device:CPU:0), values=Tensor("hash_table_Lookup:0", shape=(?,), dtype=int64, device=/device:CPU:0), dense_shape=Tensor("StringSplit:2", shape=(2,), dtype=int64, device=/device:CPU:0))


/usr/local/google/home/markdaoust/venv/local/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./checkpoints/c/model.ckpt.
INFO:tensorflow:loss = 0.0883859, step = 1
INFO:tensorflow:global_step/sec: 13.5726
INFO:tensorflow:loss = 0.0614332, step = 101 (7.366 sec)
INFO:tensorflow:global_step/sec: 13.8457
INFO:tensorflow:loss = 0.0718557, step = 201 (7.222 sec)
INFO:tensorflow:global_step/sec: 14.5126
INFO:tensorflow:loss = 0.0445167, step = 301 (6.891 sec)
INFO:tensorflow:global_step/sec: 13.6002
INFO:tensorflow:loss = 0.0446855, step = 401 (7.353 sec)
INFO:tensorflow:global_step/sec: 13.1811
INFO:tensorflow:loss = 0.0406452, step = 501 (7.587 sec)
INFO:tensorflow:global_step/sec: 13.1945
INFO:tensorflow:loss = 0.0322488, step = 601 (7.579 sec)
INFO:tensorflow:global_step/sec: 15.3108
INFO:tensorflow:loss = 0.0412634, step = 701 (6.531 sec)
INFO:tensorflow:global_step/sec: 14.2625
INFO:tensorflow:loss = 0.0385869, step = 801 (7.011 sec)
INFO:tensorflow:global_step/sec: 14.214
INFO:tensorflow

## Create and Run Experiment

In [0]:
# create experiment
def generate_experiment_fn():
  
  """
  Create an experiment function given hyperparameters.
  Returns:
    A function (output_dir) -> Experiment where output_dir is a string
    representing the location of summaries, checkpoints, and exports.
    this function is used by learn_runner to create an Experiment which
    executes model code provided in the form of an Estimator and
    input functions.
    All listed arguments in the outer function are used to create an
    Estimator, and input functions (training, evaluation, serving).
    Unlisted args are passed through to Experiment.
  """

  def _experiment_fn(output_dir):
    return tf.contrib.learn.Experiment(
        estimator,
        train_input_fn=train_input_fn,
        eval_input_fn=test_input_fn,
        train_steps=STEPS
    )
  return _experiment_fn

In [0]:
# run experiment 
learn_runner.run(generate_experiment_fn(),'ou')

## Making Predictions

In [0]:
preds = estimator.predict(input_fn=my_test_input_fn)

color_names = _get_csv_column(MY_TEST_INPUT, 'name')

print()
for p, name in zip(preds, color_names):
    color = tuple(map(int, p * 255))
    print(name, 'rgb', color)
    _plot_rgb(p)

## Pretrained model predictions

In [1]:
preds = estimator.predict(input_fn=my_test_input_fn)

color_names = _get_csv_column(MY_TEST_INPUT, 'name')

print()
for p, name in zip(preds, color_names):
    color = tuple(map(int, p * 255))
    print(name, 'rgb', color)
    _plot_rgb(p)

NameError: name 'estimator' is not defined